# WORKING WITH SUPERCOLLIDER AND OSC IN PYTHON

Explaination of how to send [OSC](http://opensoundcontrol.org/spec-1_0.html) messages from Python to [SuperCollider](https://supercollider.github.io/download) to generate audio in real time.  It assumes you have at least a passing experience with these technologies.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
import sys
sys.path.append('/Users/taube/Software/musx')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import Event, Seq, Score, Rotation, version, hertz
print(f"musx.version: {version}")

## About SuperCollider

Supercollider is a powerful synthesis program and coding environment. The SuperCollider app is built on a client-server model with two-way communication between the SuperCollider language (sclang) and it's sound server (scsynth). This communication is implemented using the OSC protocol.  The OSC protocol, in turn, makes it very easy for external environments such as Python to send audio commands to SuperCollider for realtime playback.<superscript>**</superscript>

For more information about the SuperCollider architecture the [DXARTS client/server guide to SC](http://depts.washington.edu/dxscdoc/Help/Guides/ClientVsServer.html).

![dxarts](http://depts.washington.edu/dxscdoc/Help/Guides/scEn.png)

<p style="font-size:smaller">**Sclang provides a rich coding environment for sound synthesis and composition, running Supercollider from python is not a replacement for the SC IDE!</p>

## Installation and setup

* If you don't already have SuperCollider installed on your computer, [download the app](https://supercollider.github.io/downloads.html) and follow the installation instructions.

* With your musx virtual environment activated, install the [python-osc ](https://pypi.org/project/python-osc/) package:

        (venv) $ python -m pip install python-osc

* Start the SuperCollider app and use its File menu to open the [bell.scd](./img/bell.scd) file located in the support/ subdirectory of this notebook.  This file defines an additive syntheses instrument that synthesizes sound based on spectral information taken from William A. Hibbert's terrific [website](https://www.hibberts.co.uk/lehr_1986_partial_groups/) about bell spectra.


* Once you have opened bell.scd in SuperCollider perform the two keyboard commands visible at the very top of the file: 

<pre>To execute this file type the following two keyboard commands:`
    COMMAND-A  (select all)
    COMMAND-Return  (evaluate)
</pre>        

Note: on Windows you will use Control-A and Control-Return.

    
 
 

## Sending Osc

To send musical data to SuperCollider we first create a subclass of musx's base Event class so OSC playback data can be generated to a musx Seq object. Our new class is called OscMessage and it bundles together an OSC address, a start time, and synthesis data so all will be sent to SuperCollider in a single OSC packet to be processed by SuperCollider as a bell sound. The OSC address is a string identifier for the packet, the time is the start time, and the data are the values that constitute the bell information being sent:

In [ ]:
class OscMessage(Event):
    def __init__(self, address, time, *data):
        super().__init__(time)
        self.addr = address
        self.data = [time, *data]
    def __str__(self):
        return f"<OscMessage: '{self.addr}' {self.data} {hex(id(self))}>"
    __repr__ = __str__

print(f'OscMessage: {OscMessage}')

Here is an example of an OscMessage instance:

In [ ]:
om = OscMessage("/musx", 0, 4, 220, .5)
print(f"message: {om}")
print(f"message time: {om.time}")
print(f"message address: '{om.addr}'")
print(f"message data: {[d for d in om.data]}")

Now define the function that will send OSC messages to SuperCollider in real time from a Python thread. The player accepts two inputs: a musx Seq containing the OSC messages, and an open OSC port.  The player processes the OSC messages in a loop, sending bell messages that match the current time and then waiting until the next time to play.

In [ ]:
def oscplayer(oscseq, oscout):
    messages = oscseq.events
    length = len(messages)
    thistime = messages[0].time
    nexttime = thistime
    i = 0
    while i < length:
        if messages[i].time == thistime:
            #print(f'playing {messages[i]}')
            oscout.send_message(messages[i].addr, messages[i].data)
            i += 1
            continue
        # if here then midi[i] is later than thistime so sleep
        nexttime = messages[i].time
        #print(f'waiting {nexttime-thistime}')
        time.sleep(nexttime - thistime) 
        thistime = nexttime

print(f'oscplayer: {oscplayer}')

### Composition

Define a part composer to add OSC messages to a score. This composer uses the Plain Hunt algorithm to generate the bell ringing pattern. For more information see the [Change Ringing](./changeringing.ipynb) tutorial:

In [ ]:
def plain_hunt(score, rhy, dur):
    # one descending octave of bells numbered 8, 7, ... 1
    bells = [n for n in range(8, 0, -1)]
    # dictionary of hertz values for each descending bell (D major)
    freqs = {i:f for i,f in zip(bells, hertz("d5 c# b4 a g f# e d"))}
    # Plain Hunt's rotation rules
    rules = [[0, 2, 1], [1, 2, 1]]
    # generate the Plain Hunt pattern for 8 bells
    peals = Rotation(bells, rules).all(False, True)
    # write OscMessages to the score.
    for b in peals:
        f = freqs[b]
        m = OscMessage("/musx", score.now, dur, f, .9)
        score.add(m)
        yield rhy
        
print(f'plain_hunt: {plain_hunt}')

Open an OSC output connection to SuperCollider using its default port 57120 (NetAddr.langPort in SuperCollider):

In [ ]:
import pythonosc.udp_client
import threading, time

oscout = pythonosc.udp_client.SimpleUDPClient("127.0.0.1", 57120)
print(f'oscout: {oscout}')

Generate the composition to a sequence:

In [ ]:
oscseq = Seq()   
score = Score(out=oscseq)
score.compose(plain_hunt(score, .3, 4))
print(f"oscseq: {oscseq}\nOSC messages:")
oscseq.print(end=3)
print("...")

Create a Python thread and pass it the realtime oscplayer function, the sequence and the output port. If you look in the SuperCollider window you will see printout of the score data being processed as it arrives at SuperCollider's input port:

In [ ]:
player = threading.Thread(target=oscplayer, args=(oscseq, oscout))
player.start()
print(f'Playing {oscseq}')